# Load Covid dataset

## Load CSV

In [1]:
import pandas as pd

covid_data = pd.read_csv('../lung_pollution/data/RKI_corona_landskreise.csv')

covid_data.shape

(411, 47)

In [2]:
covid_data.columns

Index(['OBJECTID', 'ADE', 'GF', 'BSG', 'RS', 'AGS', 'SDV_RS', 'GEN', 'BEZ',
       'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2', 'SN_G',
       'FK_S3', 'NUTS', 'RS_0', 'AGS_0', 'WSK', 'EWZ', 'KFL', 'DEBKG_ID',
       'Shape__Area', 'Shape__Length', 'death_rate', 'cases', 'deaths',
       'cases_per_100k', 'cases_per_population', 'BL', 'BL_ID', 'county',
       'last_update', 'cases7_per_100k', 'recovered', 'EWZ_BL',
       'cases7_bl_per_100k', 'cases7_bl', 'death7_bl', 'cases7_lk',
       'death7_lk', 'cases7_per_100k_txt', 'AdmUnitId'],
      dtype='object')

In [3]:
covid_data = covid_data[['BL','county','EWZ','Shape__Area', 'death_rate', 'cases', 'deaths','cases_per_100k']]
covid_data.head()

,BL,county,EWZ,Shape__Area,death_rate,cases,deaths,cases_per_100k
0,Schleswig-Holstein,SK Flensburg,89934,4.918293e+07,1.223721,3187,39,3543.709832
1,Schleswig-Holstein,SK Kiel,246601,1.122314e+08,1.409469,8301,117,3366.166398
2,Schleswig-Holstein,SK Lübeck,215846,2.116771e+08,1.392355,7613,106,3527.051694
3,Schleswig-Holstein,SK Neumünster,79905,7.140224e+07,0.889996,2809,25,3515.424567
4,Schleswig-Holstein,LK Dithmarschen,133251,1.425511e+09,1.915323,2976,57,2233.379112


## Feature engineering: create deaths/100k column

In [4]:
covid_data['deaths_per_100k'] = covid_data['deaths']/covid_data['EWZ']*100_000
covid_data.head()

,BL,county,EWZ,Shape__Area,death_rate,cases,deaths,cases_per_100k,deaths_per_100k
0,Schleswig-Holstein,SK Flensburg,89934,4.918293e+07,1.223721,3187,39,3543.709832,43.365134
1,Schleswig-Holstein,SK Kiel,246601,1.122314e+08,1.409469,8301,117,3366.166398,47.445063
2,Schleswig-Holstein,SK Lübeck,215846,2.116771e+08,1.392355,7613,106,3527.051694,49.109087
3,Schleswig-Holstein,SK Neumünster,79905,7.140224e+07,0.889996,2809,25,3515.424567,31.287153
4,Schleswig-Holstein,LK Dithmarschen,133251,1.425511e+09,1.915323,2976,57,2233.379112,42.776414


## Merge all Berlin 'counties' to one (to match APexpose)

Covid dataset has 11 Berlin counties, but air pollution dataset only has 1 berlin county > 
collapse 11 berlin counties into one, taking into account whether to take the sum or the mean per numerical feature

In [5]:
berlin = covid_data[covid_data["BL"] == 'Berlin']
berlin

,BL,county,EWZ,Shape__Area,death_rate,cases,deaths,cases_per_100k,deaths_per_100k
399,Berlin,SK Berlin Reinickendorf,259169,8.943665e+07,1.618289,19465,315,7510.543313,121.542314
400,Berlin,SK Berlin Charlottenburg-Wilmersdorf,315393,6.477450e+07,1.568608,20464,321,6488.412869,101.777782
401,Berlin,SK Berlin Treptow-Köpenick,272429,1.680052e+08,1.689394,13200,223,4845.299142,81.856190
402,Berlin,SK Berlin Pankow,403607,1.033630e+08,1.068934,21049,225,5215.221738,55.747299
403,Berlin,SK Berlin Neukölln,318128,4.499687e+07,1.522467,28375,432,8919.365790,135.794397
404,Berlin,SK Berlin Lichtenberg,291622,5.219800e+07,1.697550,16082,273,5514.673104,93.614336
405,Berlin,SK Berlin Marzahn-Hellersdorf,273676,6.191477e+07,1.768566,14475,256,5289.100981,93.541268
406,Berlin,SK Berlin Spandau,238922,9.294042e+07,1.444666,18551,268,7764.458694,112.170499
407,Berlin,SK Berlin Steglitz-Zehlendorf,290866,1.026872e+08,2.656722,16223,431,5577.482415,148.178199
408,Berlin,SK Berlin Mitte,374232,3.945211e+07,1.156393,30353,351,8110.744137,93.792086


In [6]:
covid_data['county'][399] = 'Berlin'
covid_data.loc[399] 

/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/360524747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['county'][399] = 'Berlin'


BL                          Berlin
county                      Berlin
EWZ                         259169
Shape__Area        89436651.129883
death_rate                1.618289
cases                        19465
deaths                         315
cases_per_100k         7510.543313
deaths_per_100k         121.542314
Name: 399, dtype: object

In [7]:
berlin_sum = berlin[['Shape__Area', 'cases', 'deaths']].sum()
berlin_sum

Shape__Area    8.933202e+08
cases          2.428130e+05
deaths         3.759000e+03
dtype: float64

In [8]:
berlin_average = berlin[['death_rate','cases_per_100k', 'deaths_per_100k']].mean()
berlin_average

death_rate            1.591636
cases_per_100k     6640.688066
deaths_per_100k     103.703091
dtype: float64

In [9]:
covid_data['cases'][399] = berlin_sum.cases
covid_data['Shape__Area'][399] = berlin_sum.Shape__Area
covid_data['deaths'][399] = berlin_sum.deaths

/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/648295955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['cases'][399] = berlin_sum.cases
/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/648295955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['Shape__Area'][399] = berlin_sum.Shape__Area
/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/648295955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [10]:
covid_data['death_rate'][399] = berlin_average.death_rate
covid_data['cases_per_100k'][399] = berlin_average.cases_per_100k
covid_data['deaths_per_100k'][399] = berlin_average.deaths_per_100k

/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/3332388361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['death_rate'][399] = berlin_average.death_rate
/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/3332388361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data['cases_per_100k'][399] = berlin_average.cases_per_100k
/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/3332388361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [11]:
covid_data.drop(index=[400,401,402,403,404,405,406,407,408,409,410], axis=0, inplace=True)

In [12]:
covid_data.shape

(400, 9)

# Load APexpose air pollution dataset

## Load data

In [13]:
import chardet
with open("../lung_pollution/data/APexpose.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.717512331990768, 'language': ''}

In [14]:
import pandas as pd 
pollution_data = pd.read_csv("../raw_data/APexpose.csv",
                             sep=';',
                             decimal='.',
                            encoding = 'Windows-1252' # needed special encoder to be able to read csv
                            )
pollution_data.head()

,county,year,NO2_annualMean,NO2_hrOver200,NO_annualMean,O3_annualMean,O3_daysOver120,O3_dailyMaxAnnualMean,O3_dailyHourlyMax,O3_daily8HrMax,PM10_annualMean,PM10_daysOver50,PM2.5_annualMean,kreis_code,scenario,ISO_code,Kreis_Scluessel,Lon,Lat
0,SK Freiburg i.Breisgau,2019,1575711,0.000000,6600048,554519,6,8370603,218.5,206925,1247323,2,8984028,12,remote,DE.BW.FB,8311,781.807.596.196.695,479.925.229.956.189
1,LK Dillingen a.d.Donau,2019,"1,89533E+14",0.004595,"9,5195E+14","5,5639E+14","3,80237E+14","7,88278E+14","1,80314E+14","1,63814E+14","1,72883E+14","5,36691E+14","1,29524E+14",68,remote,DE.BY.DD,9773,105.277.641.680.394,485.964.037.973.776
2,SK NŸrnberg,2019,2538007,0.000000,1251538,4532018,0,7369548,160.34,1556725,"1,53677E+14","4,89558E+14",1151754,107,remote,DE.BY.NR,9564,110.827.553.425.797,494.362.114.486.059
3,LK Neumarkt i.d.OPf.,2019,"1,57092E+14",0.003532,"8,1766E+14","5,20949E+14","2,18259E+14","7,65906E+14","1,79684E+14","1,62613E+14","1,60922E+14","4,89558E+14","1,22698E+14",110,remote,DE.BY.NO,9373,115.665.579.196.823,492.159.614.099.495
4,SK Rosenheim,2019,"1,75246E+14",0.004127,"8,68246E+14","5,25933E+14","2,41035E+14","7,69051E+14","1,79773E+14","1,62782E+14","1,64141E+14","4,89558E+14","1,24855E+14",122,remote,DE.BY.RH,9163,121.087.247.510.606,478.443.777.181.448


!!!!!!! Take care of hour/day format

!!!!!!! Check per pollutant which feature is most interesting

!!!!!!! PM10_daysOver50: check out exponent 14 (could be imputation error)

## Keep only 'average' scenario (combines remote and rural)

In [15]:
pollution_data.shape

(12060, 19)

In [34]:
pollution_data = pollution_data[pollution_data.scenario == 'average']
pollution_data.shape

(4020, 19)

## Keep relevant features only and sort by year

In [36]:
pollution_data.columns

Index(['county', 'year', 'NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2.5_annualMean', 'kreis_code', 'scenario',
       'ISO_code', 'Kreis_Scluessel', 'Lon', 'Lat'],
      dtype='object')

In [39]:
pollution_data = pollution_data[['county', 'year', 'NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2.5_annualMean']]

In [51]:
pollution_data.sort_values('year', 
                      axis=0, 
                      ascending=True,
                          inplace=True,
                      ignore_index=True)
pollution_data

,county,year,NO2_annualMean,NO2_hrOver200,NO_annualMean,O3_annualMean,O3_daysOver120,O3_dailyMaxAnnualMean,O3_dailyHourlyMax,O3_daily8HrMax,PM10_annualMean,PM10_daysOver50,PM2.5_annualMean
0,LK Greiz,2010,2.018928e+06,0.000000,6.925885e+06,4.277167e+06,1.682240e+14,7.595200e+13,1.795000e+14,1.622330e+14,2.428754e+06,2.200000e+01,1.850711e+06
1,LK Meißen,2010,1.686850e+05,0.000000,2.267045e+06,5.699750e+05,1.876270e+14,7.625770e+13,1.795970e+14,1.624050e+14,2.100458e+06,1.500000e+01,1.456280e+14
2,LK Erzgebirgskreis,2010,2.632646e+06,0.000000,1.539638e+06,6.527074e+06,1.627180e+14,7.586530e+14,1.794730e+14,1.621840e+14,1.183374e+06,4.000000e+00,1.381360e+14
3,LK Soest,2010,1.672297e+06,0.000000,2.669429e+06,4.640918e+06,1.836630e+14,7.619520e+14,1.795770e+14,1.623690e+14,2.017380e+14,7.341610e+14,1.733668e+06
4,LK Paderborn,2010,2.083220e+14,0.008416,1.150920e+14,5.119120e+14,1.750170e+14,7.605900e+14,1.795340e+14,1.622930e+14,1.918450e+13,5.856160e+14,1.402790e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,SK Wilhelmshaven,2019,1.063162e+06,0.000000,2.342970e+05,5.436061e+06,0.000000e+00,7.448256e+06,1.840300e+02,1.618688e+06,1.462606e+06,1.000000e+00,9.138880e+05
4016,SK Rosenheim,2019,1.800770e+14,0.006888,9.011510e+14,5.184070e+14,2.015130e+14,7.647640e+14,1.796660e+13,1.625280e+14,1.651930e+13,5.035500e+13,1.249710e+14
4017,LK Neumarkt i.d.OPf.,2019,1.625810e+14,0.005941,8.485150e+14,5.140760e+12,1.838440e+14,7.619810e+14,1.795780e+14,1.623710e+14,1.618470e+14,5.035500e+13,1.227020e+14
4018,SK Amberg,2019,1.577080e+14,0.005677,8.186720e+14,5.195000e+14,2.059690e+14,7.654670e+14,1.796880e+14,1.625670e+14,1.619050e+14,5.035500e+13,1.226390e+14


## Clean AP expose county name column to match Covid dataset

In [52]:
## Some funny characters present in county names of APExpose dataset
## Replace those characters with equivalent German character to match Covid dataset and be able to merge

pollution_data['county'] = pollution_data['county'].apply(lambda x: x.replace('Ÿ','ü'))
pollution_data['county'] = pollution_data['county'].apply(lambda x: x.replace('š','ö'))
pollution_data['county'] = pollution_data['county'].apply(lambda x: x.replace('§','ß'))
pollution_data['county'] = pollution_data['county'].apply(lambda x: x.replace('Š','ä'))

!!!!!!!!! Save as UTF-8 encoded file instead

## Convert columns to floats 

In [53]:
## some numerical features are recognized as object > convert to floats
pollution_data.dtypes

county                    object
year                       int64
NO2_annualMean           float64
NO2_hrOver200            float64
NO_annualMean            float64
O3_annualMean            float64
O3_daysOver120           float64
O3_dailyMaxAnnualMean    float64
O3_dailyHourlyMax        float64
O3_daily8HrMax           float64
PM10_annualMean          float64
PM10_daysOver50          float64
PM2.5_annualMean         float64
dtype: object

In [54]:
## Conversion to numerical feature only works if decimals are dots instead of comma > replace all commas with a dot

for column in ['NO2_annualMean', 'NO_annualMean','O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean','O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean','PM10_daysOver50', 'PM2.5_annualMean']:
    pollution_data[column] = pollution_data[column].apply(lambda x: x.replace(',','.'))
pollution_data[['NO2_annualMean', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2.5_annualMean']]

AttributeError: 'float' object has no attribute 'replace'

In [19]:
#Convert to float
for column in ['NO2_annualMean', 'NO_annualMean','O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean','O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean','PM10_daysOver50', 'PM2.5_annualMean']:
    pollution_data[column] = pollution_data[column].apply(lambda x: float(x))

In [55]:
#Check datatypes
pollution_data.dtypes

county                    object
year                       int64
NO2_annualMean           float64
NO2_hrOver200            float64
NO_annualMean            float64
O3_annualMean            float64
O3_daysOver120           float64
O3_dailyMaxAnnualMean    float64
O3_dailyHourlyMax        float64
O3_daily8HrMax           float64
PM10_annualMean          float64
PM10_daysOver50          float64
PM2.5_annualMean         float64
dtype: object

## Merge counties present in APexpore but not Covid

Eisenach (wartburgkreis), Osterode am Harz (Göttingen) present in AP dataset but not in Covid dataset (because they were merged with another county after 2019): drop those rows for all time points

In [56]:
pollution_data.shape

(4020, 13)

In [57]:
Eisenach = pollution_data[pollution_data.county == 'Eisenach']

In [72]:
Wartburgkreis = pollution_data[pollution_data.county == 'LK Wartburgkreis']

In [73]:
Wartburgkreis_merge = pd.concat(
    [Eisenach,Wartburgkreis],
    axis=0,
    ignore_index=True,
)

In [78]:
Wartburgkreis_merge.sort_values('year', 
                      axis=0, 
                      ascending=True,
                          inplace=True,
                      ignore_index=True)
Wartburgkreis_merge.columns

Index(['county', 'year', 'NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2.5_annualMean'],
      dtype='object')

In [82]:
Wartburgkreis_merge

,county,year,NO2_annualMean,NO2_hrOver200,NO_annualMean,O3_annualMean,O3_daysOver120,O3_dailyMaxAnnualMean,O3_dailyHourlyMax,O3_daily8HrMax,PM10_annualMean,PM10_daysOver50,PM2.5_annualMean
0,Eisenach,2010,1.955313e+06,0.000000,5.159791e+06,4.659175e+06,9.917384e-01,7.486420e+14,1.791560e+14,1.616200e+14,2.347870e+05,2.500000e+01,1.297480e+14
1,LK Wartburgkreis,2010,1.711520e+13,0.006405,1.009760e+14,4.884410e+14,7.926890e-01,7.455060e+14,1.790570e+14,1.614430e+13,1.708930e+14,5.035500e+13,1.281600e+14
2,Eisenach,2011,1.764411e+06,0.000000,6.108865e+06,4.483597e+06,1.032190e+14,7.492790e+14,1.791760e+14,1.616560e+14,2.188526e+06,2.300000e+01,1.307690e+14
3,LK Wartburgkreis,2011,1.723100e+14,0.006468,1.074490e+14,4.896230e+13,8.408995e-01,7.462650e+14,1.790810e+14,1.614860e+14,1.724940e+14,6.957310e+14,1.289890e+14
4,Eisenach,2012,1.711277e+06,0.000000,4.944831e+06,4.610036e+06,1.121990e+14,7.506940e+14,1.792210e+14,1.617350e+14,1.712788e+06,1.200000e+01,1.294710e+14
5,LK Wartburgkreis,2012,1.685780e+14,0.006266,9.595440e+14,4.917650e+14,9.283005e-01,7.476420e+14,1.791250e+14,1.615640e+14,1.704520e+14,6.201120e+14,1.277710e+14
6,Eisenach,2013,1.611027e+06,0.000000,9.055490e+14,4.646493e+06,0.000000e+00,7.129420e+05,1.494350e+05,1.351949e+06,1.879899e+06,1.200000e+01,1.279280e+14
7,LK Wartburgkreis,2013,1.659690e+14,0.006125,8.967950e+14,4.923560e+14,9.524044e-01,7.480220e+14,1.791370e+14,1.615850e+14,1.679110e+14,5.569320e+14,1.263730e+14
8,Eisenach,2014,1.678938e+06,0.000000,1.495335e+06,4.635543e+06,0.000000e+00,6.944509e+06,1.483027e+06,1.411733e+06,1.830367e+06,1.200000e+01,1.301900e+13
9,LK Wartburgkreis,2014,1.691190e+13,0.006295,9.508910e+14,4.907210e+14,8.856943e-01,7.469710e+14,1.791030e+14,1.615260e+14,1.714170e+14,7.251260e+14,1.283860e+14


In [81]:
Wartburgkreis_merge.groupby(['year'])['NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2.5_annualMean'].mean()


/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_38137/3502989865.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  Wartburgkreis_merge.groupby(['year'])['NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',


,NO2_annualMean,NO2_hrOver200,NO_annualMean,O3_annualMean,O3_daysOver120,O3_dailyMaxAnnualMean,O3_dailyHourlyMax,O3_daily8HrMax,PM10_annualMean,PM10_daysOver50,PM2.5_annualMean
year,,,,,,,,,,,
2010,8.557601e+12,0.003202,5.048800e+13,2.442205e+14,8.922137e-01,7.470740e+14,1.791065e+14,8.888215e+13,8.544650e+13,2.517750e+13,1.289540e+14
2011,8.615500e+13,0.003234,5.372450e+13,2.448115e+13,5.160950e+13,7.477720e+14,1.791285e+14,1.615710e+14,8.624700e+13,3.478655e+14,1.298790e+14
2012,8.428900e+13,0.003133,4.797720e+14,2.458825e+14,5.609950e+13,7.491680e+14,1.791730e+14,1.616495e+14,8.522600e+13,3.100560e+14,1.286210e+14
2013,8.298450e+13,0.003062,9.011720e+14,2.461780e+14,4.762022e-01,3.740110e+14,8.956850e+13,8.079250e+13,8.395550e+13,2.784660e+14,1.271505e+14
2014,8.455951e+12,0.003147,4.754455e+14,2.453605e+14,4.428472e-01,3.734855e+14,8.955150e+13,8.076300e+13,8.570850e+13,3.625630e+14,7.070250e+13
2015,8.232350e+13,0.003026,4.537855e+14,2.471965e+14,5.177550e+13,3.746655e+14,8.958900e+13,8.082950e+13,8.341200e+13,2.517750e+13,1.263205e+14
2016,8.330150e+13,0.003079,4.649765e+14,2.452580e+14,4.386686e-01,3.734195e+14,8.954950e+13,8.075900e+13,8.372601e+12,3.051570e+14,1.267945e+14
2017,8.213050e+13,0.003016,4.632365e+14,2.474100e+14,5.264500e+12,3.748025e+14,8.959350e+13,8.083700e+13,8.431650e+13,3.318480e+14,1.273710e+14
2018,8.184100e+13,0.003000,4.575995e+14,2.494695e+14,6.104700e+13,3.761265e+14,8.963500e+13,8.091150e+13,8.365150e+13,3.002580e+14,1.268240e+14


In [59]:
Osterode = pollution_data[pollution_data.county == 'Osterode am Harz']

In [60]:
Gottingen = pollution_data[pollution_data.county == 'LK Göttingen']

In [ ]:
#pollution_data = pollution_data[pollution_data.county != 'Eisenach']
#pollution_data.shape

In [ ]:
#pollution_data = pollution_data[pollution_data.county != 'Osterode am Harz']
#pollution_data.shape

!!!!!!!! Merge instead of dropping

In [ ]:
# air pollution data set is not organized chronologically > to extract air pollution data in chronological order, we loop over a list that indicates the correct order
time_point_order = [12,13,14, 15,16,17, 9,10,11, 3,4,5, 27,28,29, 6,7,8, 21,22,23, 18,19,20, 24,25,26, 0,1,2]
time_point_order = [i*400 for i in time_point_order] ## 400 counties total, so multiply number by 400 to get data for 1 county over time

In [ ]:
# df.sort_values(['county','year']) 
# inplace=True or assign to dataframe and check result

In [ ]:
# Retrieve data for one county (in this case number 300) and extract measurements of PM2.5 over time

temporal_time_points = [pollution_data['PM2.5_annualMean'][399+i] for i in time_point_order]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(temporal_time_points)

PROBLEM: HUGE variation in order of mangnitude for different measurements - the low ones are probably imputed > needs to be fixed!

# Merge APexpose and Covid datasets

In [ ]:
merge_df = pollution_data.merge(covid_data, on='county')
merge_df

!!!!!! Lon and Lat is not what we need - merge from Chris dataset

In [ ]:
merge_df = merge_df.rename(columns={'PM2.5_annualMean': 'PM2_5_annualMean'})

In [ ]:
merge_df.columns

In [ ]:
merge_df.dtypes

# Covid & Air pollution - quick correlation check

In [ ]:
# keep useful features only
features_latest_timepoint = merge_df.iloc[0:399][['county', 'NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2_5_annualMean']]

In [ ]:
targets_latest_timepoint = merge_df.iloc[0:399][['cases_per_100k', 'deaths_per_100k', 'deaths', 'cases']]

In [ ]:
#numerical features
feature_list = ['NO2_annualMean', 'NO2_hrOver200', 'NO_annualMean',
       'O3_annualMean', 'O3_daysOver120', 'O3_dailyMaxAnnualMean',
       'O3_dailyHourlyMax', 'O3_daily8HrMax', 'PM10_annualMean',
       'PM10_daysOver50', 'PM2_5_annualMean']

In [ ]:
target_list = ['cases_per_100k', 'deaths_per_100k', 'deaths', 'cases']

In [ ]:
#check distribution of numerical features 

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,15))

for i, feature in enumerate(feature_list):
    # First subplot
    plt.subplot(4,3,i+1)
    sns.histplot(features_latest_timepoint[feature])
    # Global figure methods
plt.suptitle('Feature distributions')
plt.show()

In [ ]:
#check distribution of targets

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,15))

for i, feature in enumerate(target_list):
    # First subplot
    plt.subplot(4,3,i+1)
    sns.histplot(targets_latest_timepoint[feature])
    # Global figure methods
plt.suptitle('Feature distributions')
plt.show()

In [ ]:
# Start a figure
plt.figure(figsize=(10,3))
# First subplot
plt.subplot(1,2,1)
plt.scatter(merge_df['NO2_annualMean'], merge_df['deaths_per_100k'])
plt.xlabel("NO2_annualMean")
plt.ylabel("deaths per 100K")
plt.title('unfiltered')
# Second subplot
plt.subplot(1,2,2) 
x = merge_df[merge_df['NO2_annualMean']>0.3*10**14]
x = x[x['NO2_annualMean']<0.3*10**15]
plt.scatter(x['NO2_annualMean'], x['deaths_per_100k'])
plt.xlabel("NO2_annualMean")
plt.ylabel("deaths per 100K")
plt.title("filtered")
# Global figure methods
plt.suptitle('NO2_annualMean')
plt.show()

PROBLEM: for each pollutant, about half of the counties don't have measurement station and instead impute values - they cluster around 0, we might want to find a better way of imputing or dropping those data

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model = smf.ols(formula = 'deaths_per_100k ~ NO2_hrOver200', data = x).fit()

In [ ]:
model.summary()

In [ ]:
# Start a figure
plt.figure(figsize=(10,3))
# First subplot
plt.subplot(1,2,1)
plt.scatter(merge_df['PM2_5_annualMean'], merge_df['deaths_per_100k'])
plt.xlabel("PM2_5_annualMean")
plt.ylabel("deaths per 100K")
plt.title('unfiltered')
# Second subplot
plt.subplot(1,2,2) 
x = merge_df[merge_df['PM2_5_annualMean']>10**14]
x = x[x['deaths_per_100k']<1200]
plt.scatter(x['PM2_5_annualMean'], x['deaths_per_100k'])
plt.xlabel("PM2_5_annualMean")
plt.ylabel("deaths per 100K")
plt.title("filtered")
# Global figure methods
plt.suptitle('PM2_5_annualMean')
plt.show()

In [ ]:
model = smf.ols(formula = 'deaths_per_100k ~ PM2_5_annualMean', data = x).fit()

In [ ]:
model.summary()